# Data and imports


In [22]:
import polars as pl
import unicodedata
from pathlib import Path
import html
import re
from tqdm.auto import tqdm  # Added tqdm import
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import nltk
from nltk.corpus import stopwords
import random
import numpy as np
from keybert import KeyBERT
import pymorphy3
import spacy

# Path
PATH_TO_ITEMS = Path().cwd().parent / "data" / "modified_data" / "items.parquet"
PATH_TO_NEW_ITEMS = Path().cwd().parent / "data" / "modified_data" / "new_items.parquet"

In [2]:
for i in range(10):
    print(15)

15
15
15
15
15
15
15
15
15
15


In [3]:
# For MacOs to check if MPS is available

print("macOS:", platform.mac_ver()[0], "  PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built‑in:", torch.backends.mps.is_built())

spacy.prefer_gpu()
print(f"spaCy GPU usage: {'Enabled' if spacy.require_gpu() else 'Disabled'}")

macOS: 15.4.1   PyTorch: 2.7.0
MPS available: True
MPS built‑in: True
spaCy GPU usage: Enabled


In [4]:
# Specifing seed value for reproducibility

SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.set_random_seed(SEED)


In [5]:
items_df = pl.read_parquet(PATH_TO_ITEMS)

# Checking if there is any null value in the DataFrame
null_sum = items_df.null_count().sum_horizontal()[0]

assert null_sum == 0, f"There are {null_sum} null values in the DataFrame"


# Preparation


## Keywords problem


- Проверим, насколько наши текущие ключевые слова подходят для кластеризации


In [6]:
# Print samples of dataset's keywords
sample_dataset = (
    items_df.sample(2)
    .select("keywords", "countries", "title", "description")
    .rows(named=True)
)

for item in sample_dataset:
    print(f"Title: {item['title']}")
    print(f"Countries: {item['countries']}")
    print(f"Keywords: {item['keywords'][:150]}...")
    print("-" * 80)

Title: с прицепом
Countries: сша
Keywords: прицепом, 2017, США...
--------------------------------------------------------------------------------
Title: виктория
Countries: великобритания
Keywords: Виктория, 2016, Великобритания, брак, короли, королевы, коррупция, отцы, дети, политика, свадьбы, семейные, проблемы, семья, отношения, отношения, муж...
--------------------------------------------------------------------------------


- Во-первых, идет дубляция страны, что не хорошо. Зачем нам заниматься увеличением токенов, которые у нас повторяются в графе Стран
- Видно, что предложения по типу "борьба за выживание" разделены запятой, а не точкой с запятой

- Необходимо придумать более осмысленные ключевые слова!


### Bert for keywords


In [7]:
# Model

# If mac, i am using Mac for this example
# device = "mps" if torch.backends.mps.is_available() else "cpu"

# If you have gpu uncomment the line below
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("cointegrated/LaBSE-en-ru", device=device)
kw_model = KeyBERT(model)  # type: ignore

In [8]:
import nltk

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")
print(f"Loaded {len(russian_stopwords)} Russian stopwords")

Loaded 151 Russian stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rustemhutiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preclean(text: str) -> str:
    text = html.unescape(text)  # &amp; → &
    text = unicodedata.normalize("NFKC", text)  # длинные тире → обычные
    text = re.sub(r"<[^>]+>", " ", text)  # убираем HTML
    text = re.sub(r"\d{4}", " ", text)  # опц.: убираем года
    text = re.sub(r"[^\S\n]+", " ", text)  # множественные пробелы
    return text.strip()


def extract_cleaned_kw(text: str) -> str:
    clean_text = preclean(text)

    extracted_keywords_with_scores = kw_model.extract_keywords(
        clean_text,  # Corrected from 'text' to 'clean_text'
        use_mmr=True,
        use_maxsum=True,
        top_n=8,
        threshold=0.35,
        stop_words=russian_stopwords,
        keyphrase_ngram_range=(1, 2),
    )

    keyword_phrases = [phrase for phrase, score in extracted_keywords_with_scores]

    return ", ".join(keyword_phrases)

In [11]:
for item in sample_dataset:
    print(f"current keywords: {item['keywords']}")
    print(f"New keywords: {extract_cleaned_kw(item['description'])}")
    print("-" * 80)

current keywords: прицепом, 2017, США
New keywords: трудности радости, актрисы фрэнки, мистер робот, молодая мама, сюжета двадцатилетняя, сериал прицепом, роль, основан личной
--------------------------------------------------------------------------------
current keywords: Виктория, 2016, Великобритания, брак, короли, королевы, коррупция, отцы, дети, политика, свадьбы, семейные, проблемы, семья, отношения, отношения, мужчины, женщины, отношения, мужа, жены, политический, конфликт, романтические, отношения, семейный, конфликт, политические, лидеры, исторические, события
New keywords: британской королевы, коулман звезда, помощником викторию, периодов существования, сериал узнать, классическим байопиком, 18, окунулась
--------------------------------------------------------------------------------


- Перепробовал кучу способов, ничего лучше не смог найти.


In [10]:
# Calculate the number of items that will be processed by extract_cleaned_kw
num_items_to_process = items_df.filter(pl.col("description") != "-").height

# Initialize tqdm progress bar
pbar = tqdm(total=num_items_to_process, desc="Extracting Keywords")


# Define a wrapper function that calls the original extract_cleaned_kw and updates the progress bar
def extract_cleaned_kw_with_pbar_update(text: str) -> str:
    result = extract_cleaned_kw(text)  # Call the original function
    pbar.update(1)
    return result


# Apply the transformation
items_df = items_df.with_columns(
    pl.when(pl.col("description") != "-")
    .then(
        pl.col("description").map_elements(
            extract_cleaned_kw_with_pbar_update, return_dtype=pl.Utf8
        )
    )
    .otherwise(pl.col("keywords"))
    .alias("new_keywords")
)

# Close the progress bar
pbar.close()

Extracting Keywords: 15963it [28:29,  9.34it/s]                           
Extracting Keywords: 15963it [28:29,  9.34it/s]


In [ ]:
# Saving our new dataset with keywords
items_df.to_parquet(
    PATH_TO_NEW_ITEMS,
)

AttributeError: 'PosixPath' object has no attribute 'child'

## Making General Description of an Item


In [6]:
# Path to item
NEW_ITEMS_PATH = Path.cwd().parent / "data" / "modified_data" / "new_items.parquet"


df_with_keywords = items_with_new_keywords = pl.read_parquet(NEW_ITEMS_PATH)

In [7]:
"Unknown" in df_with_keywords["release_year_range"].to_list()


False

In [8]:
df_with_description = df_with_keywords.with_columns(
    pl.concat_str(
        [
            # Title section
            pl.when(pl.col("title").is_not_null())
            .then(pl.lit("title: ") + pl.col("title"))
            .otherwise(None),
            # Genres section
            pl.when(pl.col("genres").is_not_null())
            .then(pl.lit("genres: ") + pl.col("genres"))
            .otherwise(None),
            # Keywords section
            pl.when(pl.col("new_keywords") != "unknown")
            .then(pl.lit("kw: ") + pl.col("new_keywords"))
            .otherwise(None),
            # Plot/Description section
            pl.when(pl.col("description") != "-")
            .then(pl.lit("plot: ") + pl.col("description"))
            .otherwise(None),
            # Directors section
            pl.when(pl.col("directors") != "unknown")
            .then(pl.lit("directors: ") + pl.col("directors"))
            .otherwise(None),
            # Actors section
            pl.when(pl.col("actors") != "unknown")
            .then(pl.lit("actors: ") + pl.col("actors"))
            .otherwise(None),
            # Studios section
            pl.when(pl.col("studios") != "unknown")
            .then(pl.lit("studios: ") + pl.col("studios"))
            .otherwise(None),
            # Release year section
            pl.when(pl.col("release_year_range").is_not_null())
            .then(pl.lit("release_year: ") + pl.col("release_year_range"))
            .otherwise(None),
            # Age rating section
            pl.when(pl.col("age_rating").is_not_null())
            .then(pl.lit("age_rating: ") + pl.col("age_rating"))
            .otherwise(None),
        ],
        separator="\n",
        ignore_nulls=True,
    ).alias("embedding_text")
)


In [9]:
print(df_with_description["embedding_text"][2])

title: тактическая сила
genres: криминал, зарубежные, триллеры, боевики, комедии
kw: капитан тейт, уайт темный, жесткую школу, навыки практике, онлайн тактическая, лишние свидетели, бандитские группировки, становятся
plot: Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказалась между двумя воюющими бандитскими группировками…  Говорят, что в спецназе нет более жестокого учителя и более профессионального бойца, чем капитан Тейт. Он заставляет своих рекрутов пройти жесткую школу, но зато после него они становятся высококлассными бойцами, которым все по плечу. Команда капитана прибывает для учений в заброшенный самолетный амбар, находящийся недалеко от города и давно превращенный в место для штатных учений. Однако вскоре оказывается, что именно здесь устраивают кровавые разборки две враждующие бандитские группировки. Лишние свидетели им ни к чему,

- Выглядит неплохо, однако все равно не нравятся ключевые слова


# Clustering


In [23]:
MODEL_NAME = "cointegrated/LaBSE-en-ru"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [24]:
texts = df_with_description["embedding_text"].to_list()
lens = []
for txt in tqdm(texts, desc="tokenize"):
    lens.append(len(tokenizer.tokenize(txt)))

tokenize:   0%|          | 0/15963 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


In [25]:
from statistics import mean, median

avg_len = mean(lens)
med_len = median(lens)
p95 = np.percentile(lens, 95)
p99 = np.percentile(lens, 99)
max_len = max(lens)

print(f"Avg len: {avg_len}")
print(f"Median len: {med_len}")
print(f"95th percentile: {p95}")
print(f"99th percentile: {p99}")
print(f"Max len: {max_len}")


Avg len: 278.7559982459438
Median len: 242
95th percentile: 487.0
99th percentile: 550.0
Max len: 2714


In [32]:
DEVICE = "mps"
MODEL_NAME = "cointegrated/LaBSE-en-ru"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()
model.max_seq_length = 512

BATCH_SHORT = 256
MAX_MODEL_TOK = 512
CHUNK_SIZE = MAX_MODEL_TOK - 2

texts = df_with_description["embedding_text"].to_list()
n_items = len(texts)


def encode_batch(batch_texts: list[str]) -> np.ndarray:
    """Encode a batch of ≤512‑token texts (no truncation needed)."""
    enc = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to(DEVICE)
    with torch.no_grad():
        out = model(**enc).last_hidden_state[:, 0]  # CLS
    emb = torch.nn.functional.normalize(out, p=2, dim=1)
    return emb.cpu().numpy().astype("float32")


def encode_long(text: str) -> np.ndarray:
    """Chunk >512‑token text into windows; mean‑pool CLS embeddings."""
    # token ids без [CLS]/[SEP]
    ids = tokenizer.encode(text, add_special_tokens=False)
    reps = []
    with torch.no_grad():
        for i in range(0, len(ids), CHUNK_SIZE):
            chunk_ids = (
                [tokenizer.cls_token_id]
                + ids[i : i + CHUNK_SIZE]
                + [tokenizer.sep_token_id]
            )
            inp = {
                "input_ids": torch.tensor([chunk_ids], device=DEVICE),
                "attention_mask": torch.ones(1, len(chunk_ids), device=DEVICE),
            }
            h = model(**inp).last_hidden_state[0, 0]  # CLS
            reps.append(h.cpu().numpy())
    vec = np.mean(reps, axis=0)
    vec = vec / np.linalg.norm(vec)
    return vec.astype("float32")

In [ ]:
embeddings = np.empty((n_items, 768), dtype="float32")

short_buffer = []
short_idx = []
for idx, txt in enumerate(tqdm(texts, total=n_items)):
    # быстрый подсчёт токенов (без полной токенизации)
    n_tok = len(tokenizer.tokenize(txt))
    if n_tok <= CHUNK_SIZE:
        short_buffer.append(txt)
        short_idx.append(idx)
        # если буфер готов отправлять
        if len(short_buffer) == BATCH_SHORT:
            embeddings[short_idx] = encode_batch(short_buffer)
            short_buffer, short_idx = [], []
    else:
        embeddings[idx] = encode_long(txt)

# не забудем хвост
if short_buffer:
    embeddings[short_idx] = encode_batch(short_buffer)

  0%|          | 0/15963 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors
